In [8]:
from PPO_mountaincar import *
import gym
import pickle


In [2]:
env = gym.make("MountainCar-v0")
agent1 = PPO(n_actions=env.action_space.n, input_dims=env.observation_space.shape)
agent1.load_models("model/best_actor_model_MC")
agent2 = PPO(n_actions=env.action_space.n, input_dims=env.observation_space.shape)
agent2.load_models("model/half_actor_model_MC")


... loading models ...
... loading models ...


In [11]:
K = 30
 
trajectory_record = [] # [(t_1,t_2)_1,...,(t_1,t_2)_K]
reward_record = []  # [(R_1,R_2)_1,...,(R_1,R_2)_K]

def softmax(x,y):
    return np.exp(x)/(np.exp(x)+np.exp(y))

for k in range(K):
    # agent1
    state, _ = env.reset()
    done = False
    cum_reward1 = 0
    cum_reward2 = 0
    trajectory_buffer1 = []  
    trajectory_buffer2 = [] 
    while not done:
        action, probs, value = agent1.choose_action(state)
        trajectory_buffer1.append([[state[0],state[1]], action]) 
        state_, reward, terminated, truncated, _ = env.step(action)
        state = state_
        cum_reward1 += reward
        done = terminated

    # agent2
    state, _ = env.reset()
    done = False

    while not done:
        action, probs, value = agent2.choose_action(state)
        trajectory_buffer2.append([[state[0],state[1]], action]) 
        state_, reward, terminated, truncated, _ = env.step(action)
        state = state_
        cum_reward2 += reward
        done = terminated

    trajectory_record.append([trajectory_buffer1,trajectory_buffer2])
    reward_record.append([cum_reward1, cum_reward2])

print(reward_record)
probs = [[softmax(x[0],x[1]),softmax(x[1],x[0])] for x in reward_record]
label = []
for prob in probs:
    label.append(0 if prob[0]>prob[1] else 1)


[[-174.0, -5468.0], [-148.0, -2989.0], [-196.0, -1521.0], [-262.0, -5819.0], [-222.0, -7469.0], [-146.0, -1933.0], [-206.0, -4689.0], [-212.0, -7138.0], [-177.0, -3573.0], [-145.0, -1527.0], [-145.0, -3141.0], [-227.0, -4056.0], [-142.0, -3683.0], [-229.0, -9512.0], [-184.0, -2267.0], [-176.0, -1427.0], [-292.0, -4408.0], [-246.0, -5789.0], [-224.0, -4585.0], [-167.0, -3153.0], [-224.0, -1802.0], [-149.0, -2252.0], [-222.0, -4305.0], [-148.0, -3989.0], [-223.0, -3991.0], [-147.0, -3249.0], [-150.0, -2417.0], [-157.0, -1472.0], [-177.0, -3756.0], [-177.0, -8531.0]]


## save traj

In [9]:
trajectory_record # K x n_policy(good one or bad one) x stepsize
label # 0 -> good one, 1 -> bad one

with open("trajectory_record.pkl", "wb") as f:
    pickle.dump({"trajectory": trajectory_record, "label": label}, f)

## load traj

In [10]:
with open("trajectory_record.pkl", "rb") as f:
    data = pickle.load(f)

trajectory_record_from_loading = data["trajectory"]
label_from_loading = data["label"]